<a href="https://colab.research.google.com/github/AndreSlavescu/Tensorflow-Projects-and-Courses/blob/main/tensorflow-projects/short_term_stockprice_preditction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
pip install pandas pandas-datareader scikit-learn yfinance

# Imports and Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import pandas_datareader.data as pdr
from datetime import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Load Stock Data


In [ ]:
# specificy start and end dates for training data
tickers = input("enter company ticker symbol: ")
start = dt(2012,1,1)
end = dt(2021,1,1)

# get data from yahoo finance using the yahoo finance api
data = pdr.DataReader(tickers, "yahoo", start, end)

# Setup datat for Neural Network

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1)) # compress stock price in a range from 0-1
scaled_data = scaler.fit_transform(data["Adj Close"].values.reshape(-1,1)) # only look at closing prices 

# define how many days you want to look into the past
pred_days = int(input("prediction days: ")) #specify prediction days

# prepare training data
x_train = []
y_train = []

for i in range(pred_days, len(scaled_data)):
  x_train.append(scaled_data[i-pred_days:i, 0]) # get next value so model can learn to predict the next day's value
  y_train.append[scaled_data[i, 0]]

x_train, y_train = np.array(x_train), np.array(y_train) # convert to numpy array

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) 

# Create the Model

In [ ]:
# create simple neural network
model = Sequential()

# specify layers - add LSTM and dropout layers
model.add(LSTM(units = 50, return_sequences = True, input_shape=(x_train.shape[1], 1))) # return data not just feed
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# final layer is one unit which is the stock price prediction
model.add(Dense(units = 1))

# Model Compilation

In [ ]:
model.compile(optimizer = "adam", loss = "mean_squared_error")

# fit the model
model.fit(x_train, y_train, epochs = 15, batche_size = 32)

model.save()

# Test Model Accuracy

In [ ]:
# Load the test data 
test_start = dt.datetime(2021, 1, 1)
test_end = dt.datetime()

test_data = pdd.DataReader(company, "yahoo", test_start, test_end) # get test data for selected company
real_prices = test_data["Close"].values # collect real closing prices of the chosen stock

total_dataset = pd.concat((data["Close"], test_data["Close"]), axis = 0)

model_inputs = total_dataset[len(total_dataset)] - len(test_data) - pred_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.transform(model_inputs)

# Predictions on Test Data

In [ ]:
x_testing = []

for i in range(pred_days, len(model_inputs)):
  x_testing.append(model_inputs[x-pred_days:x, 0])

x_testing = np.array(x_testing)
x_testing = np.reshape(x_testing, (x_testing.shape[0], x_testing.shape[1]), 1)

predictions = model.predict(x_testing)
predictions = scaler.invers_transform(predictions)

# Plot Test Predictions

In [ ]:
plt.plot(actual_prices, color = "blue", label = f"Actual Share Price for {company}")
plt.plot(predictions, color = "red", label = f"Predicted Share Price for {company}")
plt.title(f"{company} Share Price")
plt.xlabel("Variation over time")
plt.ylabel(f"{company} Share Price")
plt.legend()
plt.show()